In [5]:
from stable_baselines3 import DQN
from StockTradingEnv import StockTradingEnv
import pandas as pd
import numpy as np

# 加载数据
data = pd.read_csv('generated_stock_data.csv') # 股票历史数据，包含开盘价、收盘价等
data = data.drop(columns=['Date', 'Index'])

# 创建环境
env = StockTradingEnv(data)

# 训练 DQN
model = DQN("MlpPolicy", env, verbose=1, exploration_fraction=0.5, exploration_final_eps=0.02)
model.learn(total_timesteps=50000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


d:\ANACONDA\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.05e+03 |
|    ep_rew_mean      | 1.03e+07 |
|    exploration_rate | 0.678    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 814      |
|    time_elapsed     | 10       |
|    total_timesteps  | 8212     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.29e+06 |
|    n_updates        | 2027     |
----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 2.05e+03  |
|    ep_rew_mean      | -1.25e+03 |
|    exploration_rate | 0.356     |
| time/               |           |
|    episodes         | 8         |
|    fps              | 761       |
|    time_elapsed     | 21        |
|    total_timesteps  | 16424     |
| train/              |           |
|    learning_rate    | 0.0001    |
|    loss             | 1.37e+05  |
|    n_

In [7]:
# 测试智能体
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    print(action)
    obs, rewards, done, info = env.step(action)
    print(obs, rewards, done, info)
    if done:
        break

1
[   62.45    68.15    56.75    61.58  9115.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
1
[  129.24   130.45   128.03   128.91  1739.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
2
[  144.36   148.59   140.13   142.19  2691.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
1
[   80.8     81.04    80.56    80.85  2610.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
1
[  113.97   118.92   109.02   117.92  8647.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
1
[  128.38   135.61   121.15   132.61  1305.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
2
[   90.69    94.23    87.15    88.25  5045.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
2
[  100.27   100.62    99.92   100.06   814.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
2
[  146.73   153.66   139.8    139.85  7845.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
1
[  137.28   139.15   135.41   138.26  6331.   10000.       0.  ] 0.0 False {'总资产': 10000.0}
2
[  147.35   154.67   140.03   147.69  1299.   10000.      